In [ ]:
pip install keras-tuner

In [90]:
import keras_tuner
import keras

In [91]:
def build_model(hp):
  model = Sequential()
  model.add(Embedding(vocab_size, 25, input_length = max_length, name = "embedding"))
  
  model.add(Conv1D(filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16), kernel_size=hp.Choice('conv_1_kernel', values = [3,5]), activation = 'relu'))
  model.add(Dropout(0.2))
  model.add(MaxPooling1D(pool_size = 2))
  model.add(Conv1D(filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16), kernel_size=hp.Choice('conv_1_kernel', values = [3,5]), activation = 'relu'))
  model.add(MaxPooling1D(pool_size = 2))
  model.add(Conv1D(filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16), kernel_size=hp.Choice('conv_1_kernel', values = [3,5]), activation = 'relu'))
  model.add(MaxPooling1D(pool_size = 5))
  model.add(Conv1D(filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16), kernel_size=hp.Choice('conv_1_kernel', values = [3,5]), activation = 'relu'))
  model.add(MaxPooling1D(pool_size = 5))
  model.add(Flatten(name = 'feature_dense'))
  model.add(Dense(hp.Choice('units', [128, 64, 32, 16]), activation = 'relu'))
  model.add(Dense(hp.Choice('units', [128, 64, 32, 16]), activation = 'relu'))
  model.add(Dense(hp.Choice('units', [128, 64, 32, 16]), activation = 'relu'))
  model.add(Dense(hp.Choice('units', [128, 64, 32, 16]), activation = 'relu'))
  model.add(Dense(hp.Choice('units', [128, 64, 32, 16]), activation = 'relu'))
  model.add(Dense(1, activation = 'sigmoid'))

  model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])), loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [92]:
import tensorflow as tf
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
import keras
import re
import time
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.optimizers import Adam, SGD, Adadelta, Adagrad, RMSprop
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Flatten,Embedding,Conv1D, MaxPooling1D, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model, Model

from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sn

In [93]:
def random_upsampling(df,label):
    majority, minority = df[label].value_counts()
    df_majority = df[df[label] == 0]
    df_minority = df[df[label] == 1]

    minority_over = df_minority.sample(majority, replace = True)
    df_over = pd.concat([df_majority, minority_over], axis = 0)
    return df_over

def random_downsampling(df,label):
    majority, minority = df[label].value_counts()
    df_majority = df[df[label] == 0]
    df_minority = df[df[label] == 1]

    majority_over = df_majority.sample(minority, replace = True)
    df_over = pd.concat([df_minority, majority_over], axis = 0)
    return df_over

def preprocess(self,alist):
    for i in alist:
        split = re.split('\n',i)

        string = "\n".join([re.sub('^\d+.','',elem) for elem in split])
        string = re.split('(W)',string)
        string = [re.sub(r"[\]\//[\{\}#;.&><\\\(\)'']", ' ',x) for x in string]
        string = [x for x in string if x.strip()]
        string = " ".join(x for x in string)
        cleaned_code_snippet.append(string)
    return cleaned_code_snippet

def encode(alist, vocab_size,max_length):
    encoded_docs = [one_hot(str(d),vocab_size) for d in alist]
    padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
    return padded_docs

def find_vocab_size(alist):
    all_words = [str(w) for s in alist for w in str(s).split()]
    print(len(all_words))
    unique_words = set(all_words)
    vocab_size = len(unique_words) + 1
    return vocab_size

In [94]:
label_encoder = preprocessing.LabelEncoder()

In [95]:
df = pd.read_csv("data/training_data.csv")
#df = pd.read_csv("data/test_new.csv")
df = df[["text", "class"]]

In [96]:
df.head()

,text,class
0,hey!!,1
1,a little better what are u doing?,1
2,yea i thought u did i just woke and checked to...,1
3,so what did u do today?,1
4,sounds like fun lol,1


In [97]:
df_majority = df[df['class'] == 0]
df_minority = df[df['class'] == 1]

sample_majority = df_majority.sample(n=200000)
df = pd.concat([sample_majority, df_minority])

In [98]:
df = random_upsampling(df,"class")
#df = random_downsampling(df,"class")
# code_snippet = list(df[feature])
# df[feature]=preprocess(code_snippet)

In [99]:
len(df)

400000

In [100]:
X = df
Y = np.ravel(df[["class"]])
Y = label_encoder.fit_transform(Y)

In [101]:
X_train, X_test, y_train, y_test=train_test_split(X, Y, test_size = .20, random_state = 42)

In [102]:
docs = list(X_train["text"])

In [103]:
vocab_size = find_vocab_size(docs)
#print("vocab_size", vocab_size)
max_length=max([len(str(s).split()) for s in docs])
print("max_length", max_length)
padded_docs=encode(docs,vocab_size, max_length)

2052383
max_length 29305


In [104]:
#importing random search
from keras_tuner import RandomSearch
#creating randomsearch object
tuner = RandomSearch(build_model, objective='val_accuracy', max_trials = 10)
# search best parameter
tuner.search(padded_docs, y_train,epochs=15,validation_split=0.25)

Trial 2 Complete [00h 00m 29s]

Best val_accuracy So Far: None
Total elapsed time: 00h 01m 06s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
32                |128               |conv_1_filter
3                 |3                 |conv_1_kernel
128               |128               |units
0.01              |0.001             |learning_rate

Epoch 1/15


2024-01-27 17:27:59.936945: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 28132800000 exceeds 10% of free system memory.
2024-01-27 17:28:23.217729: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: JIT compilation failed.
2024-01-27 17:28:23.217797: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: JIT compilation failed.
Traceback (most recent call last):
  File "/home/aogundiran/miniconda3/lib/python3.11/site-packages/keras_tuner/src/engine/base_tuner.py", line 273, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/aogundiran/miniconda3/lib/python3.11/site-packages/keras_tuner/src/engine/base_tuner.py", line 238, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/aogundiran/miniconda3/lib/python3.11/site-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_va

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "/home/aogundiran/miniconda3/lib/python3.11/site-packages/keras_tuner/src/engine/base_tuner.py", line 273, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/aogundiran/miniconda3/lib/python3.11/site-packages/keras_tuner/src/engine/base_tuner.py", line 238, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/aogundiran/miniconda3/lib/python3.11/site-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/aogundiran/miniconda3/lib/python3.11/site-packages/keras_tuner/src/engine/tuner.py", line 233, in _build_and_fit_model
    results = self.hypermod

In [88]:
model=tuner.get_best_models(num_models=1)[0]
#summary of best model
model.summary()

ValueError: Received incompatible tensor with shape (110690, 25) when attempting to restore variable with shape (115367, 25) and name embedding/embeddings:0.

In [89]:
docs1 = list(X_test["text"])
padded_docs1 = encode(docs1, vocab_size, max_length)
test_preds = model.predict(padded_docs1)
test_preds = np.round(test_preds).astype(int)

test_preds1 = np.argmax(test_preds,axis=1)
accuracy = metrics.accuracy_score(y_test, test_preds)
precision = metrics.precision_score(y_test, test_preds)
recall = metrics.recall_score(y_test, test_preds)
print ("Accuracy = ", accuracy)
print ("Precision = ", precision)
print ("Recall = ", recall)

NameError: name 'model' is not defined

In [ ]:
#model.save('/home/aogundiran/Documents/New Program/models/model-01-06-24.keras')
#model.save('/home/aogundiran/Documents/New Program/models/model-01-08-24.keras')
model.save('/home/aogundiran/Documents/New Program/models/model-01-12-24.keras')

In [ ]:
# #mymodel = load_model('/home/aogundiran/Documents/New Program/models/model-01-06-24.keras')
#mymodel = load_model('/home/aogundiran/Documents/New Program/models/model-01-08-24.keras')
mymodel = load_model('/home/aogundiran/Documents/New Program/models/model-01-12-24.keras')

In [ ]:
docs1 = list(X_test["text"])
padded_docs1 = encode(docs1, vocab_size, max_length)
test_preds = mymodel.predict(padded_docs1)
test_preds = np.round(test_preds).astype(int)

test_preds1 = np.argmax(test_preds,axis=1)
accuracy = metrics.accuracy_score(y_test, test_preds)
precision = metrics.precision_score(y_test, test_preds)
recall = metrics.recall_score(y_test, test_preds)
print ("Accuracy = ", accuracy)
print ("Precision = ", precision)
print ("Recall = ", recall)

In [ ]:
vocab_dim = pd.DataFrame(list(zip([vocab_size],[max_length],)), columns=['vocab_size','max_length'])
# #vocab_dim.to_csv('/home/aogundiran/Documents/New Program/data/vocab_params.csv',index=False)
#vocab_dim.to_csv('/home/aogundiran/Documents/New Program/data/vocab_params-01-08-24.csv',index=False)
#vocab_dim.to_csv('/home/aogundiran/Documents/New Program/data/vocab_params-01-08-24-1.csv',index=False)
vocab_dim.to_csv('/home/aogundiran/Documents/New Program/data/vocab_params-01-12-24.csv',index=False)

In [ ]:
classification = classification_report(y_test, test_preds)
#print(classification.keys())
print("Classification report:" "\n", classification)

In [ ]:
# Calculate confusion matrix
confusion_mtx = confusion_matrix( y_test, test_preds, normalize= 'true')
 
# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sn.heatmap(
    confusion_mtx, xticklabels=[0,1], yticklabels=[0,1], annot=True,annot_kws={"size": 10}, cmap=sn.cubehelix_palette(as_cmap=True),linewidths=.5)
plt.xlabel("Prediction")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()